In [5]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=a722e90cda43447a54c589704c9109b6be8e0707e9d86fe44fde78b77de468ef
  Stored in directory: /root/.cache/pip/wheels/ac/62/f6/88933dadcd64a1614894614aa68cf57c4b8e5256acb650b1f1
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp39-cp39-linux_x86_64.whl size=180372 sha256=606418996bf25fdd96a869c629fd2dcd11f1c5ad7336c2fa186a565d8c8e33c6
  Stored in directory: /root/.cache/pip/wheels/8e/ff/82/9326b96f96f47472e02c453697b225813e4650c0ed4df2cd49
Successfully built nltk libwapiti
  Attempting unin

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from hazm import Normalizer, sent_tokenize,word_tokenize,Lemmatizer,Stemmer
import numpy as np
import torch
import re
import pandas as pd


In [7]:
normalizer = Normalizer()
tfidf_generator = TfidfVectorizer(ngram_range=(1, 1), norm='l2', use_idf=True, preprocessor=normalizer.normalize, tokenizer=word_tokenize)

In [8]:
hazm_normalizer = Normalizer()
link_regex = "(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"

def normalize(text: str):
    text = re.sub(r"<(/?p/?|/?br/?|)>", '', text)
    text = re.sub(link_regex, " ", text)
    text = re.sub("&?amp", " ", text)
    text = re.sub(r"[.,،?؟:;'\"/«»@!٬﷼#()<>*_]", " ", text)
    text = re.sub("[\r\n]+", ' ', text)
    text = re.sub("‌", " ", text)    # turn half-space to space in persian
    text = re.sub("(ن?می)\s+", r"\1", text) # attach prefixes of verbs to the verb
    text = hazm_normalizer.character_refinement(text)
    wierd_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u'\U00010000-\U0010ffff'
    u"\u200d"
    u"\u2640-\u2642"
    u"\u2600-\u2B55"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\u3030"
    u"\ufe0f"
    u"\u2069"
    u"\u2066"
    u"\u2068"
    u"\u2067"
    "]+", flags=re.UNICODE)
    text = wierd_pattern.sub(r'', text)
    text = re.sub("\s+", " ", text)
    return text

def separate_word(input_text):
    return len(input_text.split())

# def turn_labels_into_numbers(label: str):
#     if label == 'recommended': return 2
#     if label == 'not_recommended': return 1
#     if label == 'no_idea': return 0

In [9]:
datadf = pd.read_csv('dataWithZeroOne.csv')

In [10]:
datadf

,Unnamed: 0,زنانه/مردانه,description2,cat2,cat1,cat0,نوع لباس,قیمت,category,mainTitle,...,امکان کار در منزل,نوع دسته,تعداد شاخه,سند اداری,شاسی جلو,شاسی عقب,نوع تجهیزات,نوع تجهیزات بازی,جنس کمربند,فروشنده
0,0,1,شلوار بیرونی زنانه ،زنانه/مردانه زنانه است. ش...,لباس,کیف، کفش و لباس,وسایل شخصی,1,۷۵٬۰۰۰ تومان (مقطوع),"['وسایل شخصی', 'کیف، کفش و لباس', 'لباس', 'شلو...",شلوار بیرونی زنانه,...,0,0,0,0,0,0,0,0,0,0
1,1,0,این کالا جلو مبلی عسلی چتری است,مبلمان خانگی و میزعسلی,مبلمان و صنایع چوب,خانه و آشپزخانه,0,۲٬۹۰۰٬۰۰۰ تومان,"['خانه و آشپزخانه', 'مبلمان و صنایع چوب', 'مبل...",جلو مبلی عسلی چتری,...,0,0,0,0,0,0,0,0,0,0
2,2,0,لباس مجلسی دخترانه ،جنس غیره است. لباس مجلسی ...,جواهرات,زیورآلات و اکسسوری,وسایل شخصی,0,۱٬۱۱۱ تومان,"['وسایل شخصی', 'زیورآلات و اکسسوری', 'جواهرات'...",لباس مجلسی دخترانه,...,0,0,0,0,0,0,0,0,0,0
3,3,1,پیراهن مشکی کوتاه ترک سایز ۳۶-۳۸ ،زنانه/مردانه...,لباس,کیف، کفش و لباس,وسایل شخصی,1,۱٬۰۰۰٬۰۰۰ تومان,"['وسایل شخصی', 'کیف، کفش و لباس', 'لباس', 'پیر...",پیراهن مشکی کوتاه ترک سایز ۳۶-۳۸,...,0,0,0,0,0,0,0,0,0,0
4,4,0,این کالا ایمورتال کلاچ اتوماتیک black است,ایمورتال کلاچ اتوماتیک black,قطعات یدکی و لوازم جانبی,وسایل نقلیه,0,توافقی,"['وسایل نقلیه', 'قطعات یدکی و لوازم جانبی', 'ا...",ایمورتال کلاچ اتوماتیک black,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32723,32723,0,این کالا برقکار،برقکار ساختمان،دوربین تعمیرات ...,برقکار،برقکار ساختمان،دوربین تعمیرات,پیشه و مهارت,خدمات,0,NaN,"['خدمات', 'پیشه و مهارت', 'برقکار،برقکار ساختم...",برقکار،برقکار ساختمان،دوربین تعمیرات,...,0,0,0,0,0,0,0,0,0,0
32724,32724,0,حسابدار خانم ،ساعات کاری از ۹ تا ۱۷ پنجشنبه‌ها...,حسابدار خانم,مالی و حسابداری و حقوقی,استخدام و کاریابی (غیر رایگان),0,NaN,"['استخدام و کاریابی (غیر رایگان)', 'مالی و حسا...",حسابدار خانم,...,0,0,0,0,0,0,0,0,0,0
32725,32725,0,نیمست گردنبند طلا ورساچ E5J194 ،جنس طلا است. ...,جواهرات,زیورآلات و اکسسوری,وسایل شخصی,0,۳۸٬۲۸۰٬۰۰۰ تومان,"['وسایل شخصی', 'زیورآلات و اکسسوری', 'جواهرات'...",نیمست گردنبند طلا ورساچ E5J194,...,0,0,0,0,0,0,0,0,0,0
32726,32726,0,اجاره سوییت مبله ،متراژ ۷۰ است. اجاره سوییت م...,آپارتمان و سوئیت,اجاره کوتاه مدت,املاک,0,NaN,"['املاک', 'اجاره کوتاه مدت', 'آپارتمان و سوئیت...",اجاره سوییت مبله,...,0,0,0,0,0,0,0,0,0,0


In [11]:
data = datadf
# normalize
data["description"] = data.description.apply(normalize)
data["description2"] = data.description2.apply(normalize)


In [12]:
data = data[['description', 'description2']]

In [13]:
data

,description,description2
0,کاملا نو رنگ شتری مناسب فصل بهار و تابستان جنس...,شلوار بیرونی زنانه زنانه مردانه زنانه است شلوا...
1,تولید و پخش انواع جلو مبلی و عسلی با ۳۰٪تخفیف ...,این کالا جلو مبلی عسلی چتری است
2,همه در حد و تمیز,لباس مجلسی دخترانه جنس غیره است لباس مجلسی دخت...
3,دو عدد پیراهن بسیار شیک مشکی کوتاه سایز ۳۶-۳۸,پیراهن مشکی کوتاه ترک سایز ۳۶-۳۸ زنانه مردانه ...
4,فروش اقساطی بدون سود نصب در محل موتور بوش الما...,این کالا ایمورتال کلاچ اتوماتیک black است
...,...,...
32723,برقکاری ساختمان صنعتی و تعمیرات وبازسازی نصب ر...,این کالا برقکار برقکار ساختمان دوربین تعمیرات ...
32724,مسلط به نرم افزار هلو,حسابدار خانم ساعات کاری از ۹ تا ۱۷ پنجشنبه ها ...
32725,نیمست طلا ورساچه ۱۸عیار کد طلا E۵J۱۹۴ وزن طلا ...,نیمست گردنبند طلا ورساچ E۵J۱۹۴ جنس طلا است این...
32726,تمیز ضدعفونی شده پارکینگ دسترسی به مکان های عم...,اجاره سوییت مبله متراژ ۷۰ است اجاره سوییت مبله...


In [14]:
description_list = list(data['description'])
description2_list = list(data['description2'])
merged_description  = []
for idx in range(len(description_list)):
  merged_description.append(description_list[idx] + description2_list[idx])



In [15]:
matrix = tfidf_generator.fit_transform(merged_description)
matrix.shape

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(32728, 66900)

In [16]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
 
from gensim.models.fasttext import FastText
 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import pickle

In [17]:
word_tokens = [word_tokenize(i) for i in merged_description]

In [ ]:
embedding_size = 300
window_size = 5
min_word = 5
down_sampling = 1e-2
 
fast_Text_model = FastText(word_tokens,
                      vector_size =embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      workers = 5,
                      sg=1,
                      epochs=30)

In [ ]:
fast_Text_model.save("fasttext_model.bin")

In [ ]:
word_list = list(tfidf_generator.get_feature_names_out())
all_desc_emb = []
for idx in range(len(merged_description)):
    counter = 0
    comment_tfidf = matrix[idx]
    comment_emb = []
    for token in word_tokenize(merged_description[idx]):
        try:
            token_fast_emb = fast_Text_model.wv[token]
            token_final_emb = token_fast_emb * comment_tfidf[0 , word_list.index(token)]
            comment_emb.append(token_final_emb)
        except:
            counter+=1
            
    all_desc_emb.append(np.mean(comment_emb,axis=0))

In [36]:
df = pd.DataFrame(all_desc_emb)

In [41]:
df.to_csv('fastText.csv')

In [55]:
from numpy import array
from sklearn.decomposition import PCA
pca = PCA(80)
pca.fit(df)
pcaFastText = pca.transform(df)

In [51]:
pcaFastText.shape

(32728, 50)

In [1]:
import pickle
with open("labels", "rb") as fp:   # Unpickling
  labels = pickle.load(fp)

In [3]:
from sklearn.model_selection import train_test_split
X_train , X_test , Y_train , Y_test = train_test_split(pcaFastText, labels , test_size = 0.2 )

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
classifier = LogisticRegression(max_iter =10000)
classifier.fit(X_train , Y_train)
Y_pred = classifier.predict(X_test)
acc = accuracy_score(Y_test , Y_pred)
print(acc)

0.2283837457989612


In [59]:
import pandas as pd
column_values = []
for i in range(80):
  column_values.append("pcaFastText" + str(i))

dfpcaFastText = pd.DataFrame(data = pcaFastText, 
                  columns = column_values)
dfpcaFastText.to_csv("pcaFastText.csv")

In [60]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=3, learning_rate='auto',init='random', perplexity=30 ,  random_state=30).fit_transform(pcaFastText)

In [62]:
X_embedded.shape

(32728, 3)

In [63]:
from sklearn.model_selection import train_test_split
X_train , X_test , Y_train , Y_test = train_test_split(X_embedded, labels , test_size = 0.2 )
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
classifier = LogisticRegression(max_iter =10000)
classifier.fit(X_train , Y_train)
Y_pred = classifier.predict(X_test)
acc = accuracy_score(Y_test , Y_pred)
print(acc)

0.6782768102658112


In [64]:
import pandas as pd
column_values = []
for i in range(3):
  column_values.append("TSNEFastText" + str(i))

dfTSNEFastText = pd.DataFrame(data = X_embedded, 
                  columns = column_values)
dfTSNEFastText.to_csv("TSNEFastText.csv")